In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [4]:
""" llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head() """

' llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"\nvllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)\nvllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)\nvllamadaEmpresa.head() '

vllamadaEmpresa.shape


In [5]:
# Recuperar empresas ID de salesforce desde Account
""" results = sf.query_all("""
    #"""
  #SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    #"""
""")
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape """

')\nEmpresas_SF = pd.DataFrame(results["records"])\nEmpresas_SF = Empresas_SF[\n    [\n        "Id",\n        "Name",\n        "Holding__c",\n        "Campa_a__c",\n        "Id_Empresa__c",\n        "Empleador__c",\n        "poliza__c",\n        "RUT_Empresa__c",\n    ]\n]\nEmpresas_SF.shape '

In [6]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
#results = sf.query_all(
#    """
#   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
#    """
#)
#Programas_por_empresa_SF = pd.DataFrame(results["records"])

#Programas_por_empresa_SF = Programas_por_empresa_SF[
#    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
#]
#Programas_por_empresa_SF.shape

In [7]:
# Union de Programas Empresa y empresas
""" Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape """

' Programas_empresas = pd.merge(\n    left=Empresas_SF,\n    right=Programas_por_empresa_SF,\n    how="left",\n    left_on="Id",\n    right_on="Empresa__c",\n)\nProgramas_empresas = Programas_empresas.rename(\n    columns={\n        "Id_x": "Id Account Empresa",\n        "Name_x": "Nombre Empresa",\n        "Holding__c": "Holding",\n        "Campa_a__c": "Campaña",\n        "Empleador__c": "Empleador",\n        "poliza__c": "Poliza",\n        "RUT_Empresa__c": "Rut Empresa",\n        "Id_y": "Programas_por_Empresas__c",\n        "Id_Empresa__c": "Codigo EMP",\n        "Name_y": "Codigo A",\n    }\n)\nProgramas_empresas = Programas_empresas[\n    [\n        "Id Account Empresa",\n        "Nombre Empresa",\n        "Holding",\n        "Campaña",\n        "Empleador",\n        "Poliza",\n        "Rut Empresa",\n        "Programas_por_Empresas__c",\n        "Codigo EMP",\n        "Codigo A",\n    ]\n]\n\nProgramas_empresas.shape '

In [8]:
#Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [9]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.69s/it]


,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,False,a3yRN0000006mmjYAA,775173998Sin Póliza Asociada,1
1,-,Apunto Servicios De Alimentacion,Apunto Servicios De Alimentacion,-,Consalud,Holding Consalud,-,Chile,False,a3yRN0000006rG9YAI,-,1
2,-,2M Inmobiliaria Spa.,2M Inmobiliaria Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGAYAY,-,1
3,-,360 Gms Capacitacion Spa.,360 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGBYAY,-,1
4,-,361 Gms Capacitacion Spa.,361 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGCYAY,-,1


fin empresas y comienzo de rescate de pacientes


In [10]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId": "Id Contacto",
    }
)
Pacientes_SF.shape

(2001448, 10)

In [11]:
def siPrimerCero(y,campana):
    if campana != "ProChile":    
        if len(y) <= 4:
            return "NONE"
        if y[0] == "0":
            y = y[1:]
        #return y
    return y

In [12]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y,"0"))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16168\933068120.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")


In [13]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE")]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE")]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas\_\_c relaciones de empresas con pacientes

results = sf.query_all(
"""
select Id,Paciente**c, Programas_por_Empresas**c,padre**c from Paciente_Programas**c  
 """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
columns={
"Id": "Id Pacientes_Programas**c",
"Paciente**c": "Id Account",
"Programas_por_Empresas**c": "Programas_por_Empresas**c",
"padre\_\_c": "Nombre Campaña",
}
)
Paciente_Programas_SF.shape


# Union de Pacientes Programas y Programas por empresa

empresas_unidas = pd.merge(
left=Paciente_Programas_SF,
right=Programas_empresas,
how="left",
left_on="Programas_por_Empresas**c",
right_on="Programas_por_Empresas**c",
)
empresas_unidas.shape


Carga de Archivo carga


In [33]:
#nuevos = "F:/Empresas SF/reportes/SFTP.xlsx"
#nuevos = "F:/Cargas Base Datos CRM/MetLife/General (Servidor Interno)/CUADRATURA/Meses/Metlife 040324.xlsx"
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife Agencia"
Pais = "Chile"
baseFinal = True
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,poliza,rutE,empleador,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,rut1,...,direccion,comuna,ciudad,celular,email,nacimiento,sexo,VIGENCIA,VIGENCIADESDE,VIGENCIAHASTA
0,123240000001,B,MetLife Joven Premium,GERVACIO AARON,RODRIGUEZ,PEÑA,RODRIGUEZ PEÑA,19333177,7,19333177-7,...,Valle Trancura 1235,CONCEPCION,CONCEPCION,57658381.0,rodriguezgervacio@gmail.com,1996-05-01,M,VIGENTE,2024-01-04,2025-01-04
1,123240000001,B,MetLife Joven Premium,CATALINA ANDREA,MANRÍQUEZ,NÚÑEZ,MANRÍQUEZ NÚÑEZ,20197475,5,20197475-5,...,Valle Trancura 1235,CONCEPCION,CONCEPCION,975171632.0,cata.manriquezn@gmail.com,2000-04-07,F,VIGENTE,2024-01-04,2025-01-04
2,123240000002,B,MetLife Joven Premium,MAURICIO JAIME,HIDALGO,VELASQUEZ,HIDALGO VELASQUEZ,15441551,3,15441551-3,...,Camino Al Volcan/ Las Viscachas 05879 Cod 3/ C-9,PUENTE ALTO,SANTIAGO,89019246.0,mauricio.hidalgo.v@gmail.com,1982-09-14,M,VIGENTE,2024-01-04,2025-01-04
3,123240000003,B,MetLife Joven,MARCIA ISABEL,SOTO,ANTILEF,SOTO ANTILEF,18848699,1,18848699-1,...,MARZOtinez De Rosas 0362,LONCOCHE,LONCOCHE,84557671.0,flga.msoto@gmail.com,1994-09-23,F,VIGENTE,2024-01-04,2025-01-04
4,123240000004,B,MetLife Joven,MARIA JOSE,SALINAS,CAMPUSANO,SALINAS CAMPUSANO,16557810,4,16557810-4,...,Psj El Tulipero 3690,MACUL,SANTIAGO,8888081.0,salinasc.majo@gmail.com,1987-07-09,F,VIGENTE,2024-01-04,2025-01-04


In [34]:
Pacientes_Archivo.shape

(698, 21)

In [35]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",
        #"Id Contacto",
    ]
]

In [36]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("-", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("–", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(" ", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.upper()
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.strip()
Pacientes_Archivo.shape

Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].astype(str)
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.upper()
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("-", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("–", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\1490231206.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\1490231206.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\1490231206.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["ru

(698, 22)

In [37]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\2296516763.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(698, 23)

In [38]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┐", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("═", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("╔", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┌", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┐", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("═", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("╔", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┌", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\3410721566.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\3410721566.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [39]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y,campaña))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y,campaña))
#Pacientes_Archivo

# Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
# Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False)

In [40]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE")]
# | (Pacientes_Archivo["rutT"] != "NONE")


In [ ]:
Pacientes_Archivo["Llave"] = Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]
Pacientes_Archivo["PolizaLlave"] = Pacientes_Archivo["poliza"] +  Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]

#Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/eliminar2.xlsx", index=False)

PAcientes Programas


In [ ]:
""" query_pacientePrograma = (
    "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '"
    + campaña
    + "'"
)
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape
PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head() """

account contact relation


In [ ]:
""" query_accountContact = (
    "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '"
    + campaña
    + "'"
)
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId": "Id Account Paciente",
    }
)
accountContactRelation.head()
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head() """

empresa contacto convenio


In [41]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc, Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c, Empresa__c, Paciente__c,N_Documento_Titular__c, Relaci_n_Con_Asegurado__c, Identificador__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [00:05<00:00,  5.75s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__c,Paciente__c,N_Documento_Titular__c,Relaci_n_Con_Asegurado__c,Identificador__c
0,a40RN0000019a6vYAA,132416842,-,-,a3yRN0000006tT9YAI,001RN000000qos9YAA,132416842,Titular,MetLife A.
1,a40RN0000019ex8YAA,257966194,-,-,a3yRN0000006tT9YAI,001RN0000069HW1YAM,257966194,Titular,MetLife A.
2,a40RN000001BqCPYA0,13996330K,-,-,a3yRN0000006tT9YAI,001RN000000qp6kYAA,13996330K,Titular,MetLife A.
3,a40RN000001Fc4PYAS,187659965,-,-,a3yRN0000006tT9YAI,001RN000000qohnYAA,187659965,Titular,MetLife A.
4,a40RN000001Fc61YAC,187658012,-,-,a3yRN0000006tT9YAI,001RN000006vHqKYAU,187659965,Carga,MetLife A.


In [42]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c": "Rut Titular",
        "Empresa__r.P_liza__c":"Poliza",
        "Empresa__r.Rut_Empresa__c":"Rut Empresa",
        "Relaci_n_Con_Asegurado__c":"Relación",
        "Identificador__c":"Nombre Empresa",
        "Campa_a__c":"Campaña",
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Relación,Nombre Empresa
0,a40RN0000019a6vYAA,132416842,-,-,a3yRN0000006tT9YAI,001RN000000qos9YAA,132416842,Titular,MetLife A.
1,a40RN0000019ex8YAA,257966194,-,-,a3yRN0000006tT9YAI,001RN0000069HW1YAM,257966194,Titular,MetLife A.
2,a40RN000001BqCPYA0,13996330K,-,-,a3yRN0000006tT9YAI,001RN000000qp6kYAA,13996330K,Titular,MetLife A.
3,a40RN000001Fc4PYAS,187659965,-,-,a3yRN0000006tT9YAI,001RN000000qohnYAA,187659965,Titular,MetLife A.
4,a40RN000001Fc61YAC,187658012,-,-,a3yRN0000006tT9YAI,001RN000006vHqKYAU,187659965,Carga,MetLife A.


In [43]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(" ", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()

empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("-", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("–", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(" ", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.upper()
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.strip()

empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("-", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("–", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(".", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(" ", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.upper()
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.strip()


empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\3330564967.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\3330564967.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\3330564967.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when r

(793, 9)

In [44]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Relación,Nombre Empresa,Rut_y
0,a40RN0000019a6vYAA,132416842,,,a3yRN0000006tT9YAI,001RN000000qos9YAA,132416842,Titular,MetLife A.,132416842
1,a40RN0000019ex8YAA,257966194,,,a3yRN0000006tT9YAI,001RN0000069HW1YAM,257966194,Titular,MetLife A.,257966194
2,a40RN000001BqCPYA0,13996330K,,,a3yRN0000006tT9YAI,001RN000000qp6kYAA,13996330K,Titular,MetLife A.,13996330K
3,a40RN000001Fc4PYAS,187659965,,,a3yRN0000006tT9YAI,001RN000000qohnYAA,187659965,Titular,MetLife A.,187659965
4,a40RN000001Fc61YAC,187658012,,,a3yRN0000006tT9YAI,001RN000006vHqKYAU,187659965,Carga,MetLife A.,187658012


In [ ]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

Rescatamos todos los ruts Id con su empresa desde la campaña


In [45]:
#Pacientes_Archivo.head()

Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)


Pacientes_Archivo.drop_duplicates(
[
"rut",
"rutT",
"LLaveEmpresa",
"LLAveRut",
],
keep="last",
)
Pacientes_Archivo.shape


In [46]:
print(campaña)

MetLife Agencia


In [47]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [48]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(698, 34)

In [49]:
#Pacientes_con_id.head()
#Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [50]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        print("date_string =", date_string)
        print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        print("date_object =", date_object)
        print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    

In [51]:
noestan = pd.DataFrame()
if ("nacimiento" in noestan.columns) == True:  
    Pacientes_con_id["dertyuiop"] = Pacientes_con_id["nacimiento"]
    Pacientes_con_id["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
    Pacientes_con_id.head()
noestan = Pacientes_con_id[(Pacientes_con_id["RutEstaoNo"] == False)]

In [52]:
#noestan.to_excel("F:/Empresas SF/reportes/noestan.xlsx", index=False)
noestan.head()

,poliza,rutE,empleador,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,rut1,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
626,123240000585,B,MetLife Joven,Ignacia Fernanda,MANSILLA,PINTO,Mansilla Pinto,18324203,2,18324203-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,123240000586,B,MetLife Joven,Diego Mario,BOTTO,GONZALEZ,Botto Gonzalez,19437943,9,19437943-9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,123240000587,B,MetLife Joven,Rodrigo,ORTIZ,CHAPARRO,Ortiz Chaparro,19293897,K,19293897-K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630,123240000589,B,MetLife Joven +,Paulina Belén,SOTO,LARA,Soto Lara,19782876,5,19782876-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,123240000590,B,MetLife Joven +,Erica,ALIAGA,MIRANDA,Aliaga Miranda,11277562,5,11277562-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:


# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# Subir_pacientes["Email_Seteado__c"] = ""
#Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]

# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Subir_pacientes["rut1"] = noestan["rut"]
Subir_pacientes["rut2"] = noestan["rutT"]

if ("nombres" in noestan.columns) == True:
    Subir_pacientes["FirstName"] = noestan["nombres"]
if ("apellidos" in noestan.columns) == True:
    Subir_pacientes["LastName"] = noestan["apellidos"]
if ("rut" in noestan.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        noestan["rut"].astype(str).str.upper()
    )


if ("email" in noestan.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = noestan["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
#if ("nacimiento" in Pacientes_con_id.columns) == True:    
#    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"].astype(str), format="%Y-%m-%d")  
if ("nacimiento" in noestan.columns) == True:    
    #try:
    Subir_pacientes["PersonBirthdate"] = noestan["nacimiento"].apply(lambda y: fechaValida(y))
    Subir_pacientes["derty"] = noestan["nacimiento"].apply(lambda y: fechaValida(y))
    #except ValueError:
        #Subir_pacientes["PersonBirthdate"] = pd.to_datetime("1900-01-01", format="%Y-%m-%d")
 
if ("sexo" in noestan.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    noestan.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = noestan["sexo"]
if ("direccion" in noestan.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        noestan["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in noestan.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        noestan["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in noestan.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        noestan["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = noestan["cargo"].str.title()

if ("division" in noestan.columns) == True:
    Subir_pacientes["PersonDepartment"] = noestan["division"].str.title()

if ("celular" in noestan.columns) == True:
    Subir_pacientes["Phone"] = (
        noestan["celular"].astype(str).str.strip().replace("+", "")
        
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"

#Subir_pacientes_Que_no_Estan = Subir_pacientes
#Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
#Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]

#Subir_pacientes["PersonBirthdate"] = Subir_pacientes["PersonBirthdate"].str.replace("0001-01-01 00:00:00", "")

# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)
Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
)
Subir_pacientes.head()

date_string = 1991-12-30 00:00:00
type of date_string = <class 'str'>
date_object = 1991-12-30 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1996-05-20 00:00:00
type of date_string = <class 'str'>
date_object = 1996-05-20 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1995-12-15 00:00:00
type of date_string = <class 'str'>
date_object = 1995-12-15 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1997-09-16 00:00:00
type of date_string = <class 'str'>
date_object = 1997-09-16 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1968-07-07 00:00:00
type of date_string = <class 'str'>
date_object = 1968-07-07 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1997-10-02 00:00:00
type of date_string = <class 'str'>
date_object = 1997-10-02 00:00:00
type of date_object = <class 'datetime.datetime'>
date_string = 1987-12-27 00:00:00
type of date_string = <class 'str'>
date_o

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\2697898823.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noestan.replace({"sexo": Sexo}, inplace=True)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\2697898823.py:97: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19472\2697898823.py:104: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as l

,rut1,rut2,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
626,183242032,183242032,Ignacia Fernanda,Mansilla Pinto,183242032,ignacia.mansillap@gmail.com,True,1991-12-30,1991-12-30,Mujer,volcan corcovado 5586,Puerto Montt,Puerto Montt,6617915,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
627,194379439,194379439,Diego Mario,Botto Gonzalez,194379439,diegobotto.gon@gmail.com,True,1996-05-20,1996-05-20,Hombre,"camino el cerrillo 2375, casa 72. linderos",Buin,Santiago,5799718,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
628,19293897K,19293897K,Rodrigo,Ortiz Chaparro,19293897K,rodrigo.ortiz.chaparro@gmail.com,True,1995-12-15,1995-12-15,Hombre,diagonal paraguay 406 dp 1207,Santiago,Santiago,72133997,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
630,197828765,197828765,Paulina Belén,Soto Lara,197828765,paulina.sotolara@gmail.com,True,1997-09-16,1997-09-16,Mujer,av el parrón 0677 condominio don arturo depto 102,La Cisterna,Santiago,66676533,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
631,112775625,112775625,Erica,Aliaga Miranda,112775625,erikaaliaga69@gmail.com,True,1968-07-07,1968-07-07,Mujer,camino del amanecer 1915,Concepcion,Concepcion,9967716,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [54]:
Subir_pacientes.shape

(59, 19)

In [55]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc where IdentificationId__pc = '" + rut + "')"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    #vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9] ,
                        #"PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
   

In [56]:
Subir_pacientes.head()

,rut1,rut2,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
626,183242032,183242032,Ignacia Fernanda,Mansilla Pinto,183242032,ignacia.mansillap@gmail.com,True,1991-12-30,1991-12-30,Mujer,volcan corcovado 5586,Puerto Montt,Puerto Montt,6617915,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
627,194379439,194379439,Diego Mario,Botto Gonzalez,194379439,diegobotto.gon@gmail.com,True,1996-05-20,1996-05-20,Hombre,"camino el cerrillo 2375, casa 72. linderos",Buin,Santiago,5799718,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
628,19293897K,19293897K,Rodrigo,Ortiz Chaparro,19293897K,rodrigo.ortiz.chaparro@gmail.com,True,1995-12-15,1995-12-15,Hombre,diagonal paraguay 406 dp 1207,Santiago,Santiago,72133997,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
630,197828765,197828765,Paulina Belén,Soto Lara,197828765,paulina.sotolara@gmail.com,True,1997-09-16,1997-09-16,Mujer,av el parrón 0677 condominio don arturo depto 102,La Cisterna,Santiago,66676533,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
631,112775625,112775625,Erica,Aliaga Miranda,112775625,erikaaliaga69@gmail.com,True,1968-07-07,1968-07-07,Mujer,camino del amanecer 1915,Concepcion,Concepcion,9967716,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [57]:
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)


Subir_pacientes.shape

(59, 19)

In [ ]:
""" Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
) """

In [58]:
""" Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]
# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()
Subir_pacientes.shape """

' Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)\nSubir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]\n# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()\nSubir_pacientes.shape '

ACA SE CREAN LOS PACIENTES Q NO ESTAN

In [59]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)


In [ ]:
""" Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
) """
#Subir_pacientes["Id_creado"]

In [60]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c WHERE Empresa_Padre__r.Name = '" + campaña + "' order by CreatedDate"

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]

df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_asociadas.xlsx", index=False)
df2.head()
df2.shape

100%|██████████| 1/1 [00:04<00:00,  4.66s/it]


(2, 12)

In [61]:
#RELACION EMPRESA PACIENMTES ---- EMPRESACONTACTOCONVENIOS


sinId = pd.DataFrame()
Subir_pacientes["Id_creado"] = Subir_pacientes["Id_creado"].astype(str)
sinId = Subir_pacientes[Subir_pacientes["Id_creado"].astype(str).str.len() < 12]
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id_creado"].astype(str).str.len() >= 12]
sinId.shape

(0, 21)

In [62]:
sinId.head()

,rut1,rut2,FirstName,LastName,IdentificationId__pc,PersonEmail,Email_Seteado__c,PersonBirthdate,derty,HealthCloudGA__Gender__pc,...,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,dato_account,Id_creado


In [63]:
sinId["Id_creado"] = sinId["rut1"].apply(lambda y: traerId(y))

Subir_pacientes = pd.concat([Subir_pacientes, sinId], axis=0)
Subir_pacientes.shape

(59, 21)

In [64]:
""" Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/dertyuiop.xlsx", index=False
)
Subir_pacientes.head() """

' Subir_pacientes.to_excel(\n    "F:/Empresas SF/reportes/dertyuiop.xlsx", index=False\n)\nSubir_pacientes.head() '

In [65]:
Subir_pacientes = Subir_pacientes[
    [
        "rut1",
        "rut2",
        "Id_creado",
        
    ]
]





In [66]:
Subir_pacientes.shape

(59, 3)

In [67]:
Pacientes_con_id["Id_creado"] = Pacientes_con_id["Id Account Paciente"]

In [68]:
Pacientes_con_id.shape

(698, 35)

In [69]:

#noestan["Id Account Paciente"] = np.where(Subir_pacientes["IdentificationId__pc"] == noestan["rut"],Subir_pacientes["Id_creado"] + "SI","NO")
todos_con_Id = pd.merge(
    left=Pacientes_con_id,
    right=Subir_pacientes,
    how="left",
    left_on="rut",
    right_on="rut1",
)

todos_con_Id = todos_con_Id[
    [
        "poliza",
        #"empleador",
        "rutE",
        "rut",
        "rutT",
        "Id_creado_x",
        "Id_creado_y",
    ]
]

todos_con_Id.shape


(698, 6)

In [70]:
todos_con_Id.head()

,poliza,rutE,rut,rutT,Id_creado_x,Id_creado_y
0,123240000001,B,193331777,193331777,001RN000006kbAtYAI,NaN
1,123240000001,B,201974755,201974755,001RN000006kW9XYAU,NaN
2,123240000002,B,154415513,154415513,001RN000006kcdDYAQ,NaN
3,123240000003,B,188486991,188486991,001RN000006kcwbYAA,NaN
4,123240000004,B,165578104,165578104,001RN000006kdpLYAQ,NaN


In [71]:
todos_con_Id.Id_creado_x.fillna(todos_con_Id["Id_creado_y"], inplace=True)
todos_con_Id.Id_creado_x.fillna("FARSO", inplace=True)
todos_con_Id = todos_con_Id[todos_con_Id["Id_creado_x"] != "FARSO"]
todos_con_Id2 = todos_con_Id[todos_con_Id["Id_creado_x"] == "FARSO"]
todos_con_Id2["Id_creado_x"] = todos_con_Id2["rut"].apply(lambda y: traerId(y))
todos_con_Id = pd.concat([todos_con_Id, todos_con_Id2], axis=0)

In [72]:
todos_con_Id.head()

,poliza,rutE,rut,rutT,Id_creado_x,Id_creado_y
0,123240000001,B,193331777,193331777,001RN000006kbAtYAI,NaN
1,123240000001,B,201974755,201974755,001RN000006kW9XYAU,NaN
2,123240000002,B,154415513,154415513,001RN000006kcdDYAQ,NaN
3,123240000003,B,188486991,188486991,001RN000006kcwbYAA,NaN
4,123240000004,B,165578104,165578104,001RN000006kdpLYAQ,NaN


In [73]:
""" todos_con_Id.to_excel(
    "F:/Empresas SF/reportes/todos_con_Id.xlsx", index=False
) """


' todos_con_Id.to_excel(\n    "F:/Empresas SF/reportes/todos_con_Id.xlsx", index=False\n) '

In [74]:
if campaña == "Southbridge Plan Mujer":
    VempresaConvenio = "a3yRN0000006sVTYAY"
elif campaña == "Southbridge":
    
    VempresaConvenio = "a3yRN0000006sqYYAQ"
elif campaña == "Construmart":
    
    VempresaConvenio = "a3yRN0000006rlZYAQ"
elif campaña == "EY":
    
    VempresaConvenio = "a3yRN0000006rsTYAQ"
elif campaña == "Colmena":

    VempresaConvenio = "a3yRN0000006rf5YAA"
elif campaña == "Pedro De Valdivia":
    
    VempresaConvenio = "a3yRN0000006sVzYAI"
elif campaña == "Forum":
    
    VempresaConvenio = "a3yRN0000006s3SYAQ"
elif campaña == "Farmacias Cruz Verde":
    
    VempresaConvenio = "a3yRN0000006rsqYAA"
elif campaña == "Milan Fajanovic":
    
    VempresaConvenio = "a3yRN0000006sR4YAI"
elif campaña == "ProChile":
    
    VempresaConvenio = "a3yRN0000006rpsYAA"
elif campaña == "Farmacias Ahumada":
    
    VempresaConvenio = "a3yRN0000006rspYAA"
elif campaña == "Minera El Abra":
    
    VempresaConvenio = "a3yRN0000006snOYAQ"
elif campaña == "Instruvalve":
    
    VempresaConvenio = "a3yRN0000006s9DYAQ"
elif campaña == "VidaCel":
    
    VempresaConvenio = "a3yRN0000006suzYAA"
elif campaña == "Care Assistance Colaboradores":
    
    VempresaConvenio = "a3yRN0000006reXYA"
elif campaña == "Nexo":
    
    VempresaConvenio = "a3yRN0000006sORYAY"
elif campaña == "Talo":
    
    VempresaConvenio = "a3yRN0000006t10YAA"
elif campaña == "MetLife Rentas Vitalicias":
    
    VempresaConvenio = "a3yRN0000006tWiYAI"
elif campaña == "Cortesía MetLife Orienta":
    
    VempresaConvenio = "a3yRN0000008OLJYA2"
else:
    VempresaConvenio = "Buscarv"


In [75]:
if(campaña == "MetLife" or campaña == "BCI Seguros Colectivos"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_x"]
    todos_con_Id["llave"] = todos_con_Id["poliza"] + todos_con_Id["rutT"] + todos_con_Id["rut"]
    
elif(campaña == "Vida Security"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] +empresacontactoconvenio_PacientesSF["Rut Empresa"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_x"]
    todos_con_Id["llave"] = todos_con_Id["poliza"] + todos_con_Id["rutE"] + todos_con_Id["rutT"] + todos_con_Id["rut"]
else:
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_x"]
    todos_con_Id["llave"] = todos_con_Id["rutT"] + todos_con_Id["rut"]    
    

todos_con_Id["esta"] = todos_con_Id["llave"].isin(empresacontactoconvenio_PacientesSF["llave"])
empresacontactoconvenio_PacientesSF["Noesta"] = empresacontactoconvenio_PacientesSF["llave"].isin(todos_con_Id["llave"])


diccionario = df2[
    [
        "P_liza__c",
        "Id",        
    ]
]
diccionario = diccionario.drop_duplicates(
    [
        "P_liza__c",
        "Id",        
    ],
    keep="last",
)
diccionario = diccionario.drop_duplicates(
    [
        "P_liza__c",        
    ],
    keep="last",
)

diccionario = diccionario.set_index("P_liza__c").T.to_dict(orient="list")

diccionario2 = df2[
    [
        "P_liza__c",
        "Activo__c",        
    ]
]
diccionario2 = diccionario2.drop_duplicates(
    [
        "P_liza__c",        
    ],
    keep="last",
)

diccionario2 = diccionario2.set_index("P_liza__c").T.to_dict(orient="list")


todos_con_Id["N_Documento_Titular__c"] = todos_con_Id["rutT"]
todos_con_Id["Paciente__c"] = todos_con_Id["Id_creado_x"]
todos_con_Id["Empresa__c"] = VempresaConvenio
todos_con_Id["EmpActiva"] = "revisar"


todos_con_Id = todos_con_Id[
    [
        "poliza",
        "rutE",
        "rut",
        "rutT",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
        "esta",
        "EmpActiva",
    ]
]


if (VempresaConvenio == "Buscarv"):
    todos_con_Id["poliza2"] = todos_con_Id["poliza"] 
    todos_con_Id.replace({"poliza2": diccionario}, inplace=True)    
    todos_con_Id["Empresa__c"] = np.where(todos_con_Id["poliza2"].str.len() >= 15, todos_con_Id["poliza2"], todos_con_Id["Empresa__c"])
    todos_con_Id["EmpActiva"] = todos_con_Id["poliza"] 
    todos_con_Id.replace({"EmpActiva": diccionario2}, inplace=True)
    
todos_con_Id = todos_con_Id[
    [
        "poliza",        
        "rutE",
        "rut",
        "rutT",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
        "esta",
        "EmpActiva",
    ]
]


In [ ]:
todos_con_Id.head(50)

In [76]:

todos_con_Id = todos_con_Id[todos_con_Id["esta"] == False]


todos_con_Id.to_excel(
    "F:/Empresas SF/reportes/todos con.xlsx", index=False
)

In [77]:

empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/empresacontacto.xlsx", index=False
)

Cuadrar eliminar pacientes q no estan en el listado
//-----------------------------------------------------------//


In [ ]:
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "163820064")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "153298939")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "191710940")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "182470082")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "HEGJ930928HMNRRL06")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16007258K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "86676540")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "202201466")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "105233469")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "130949185")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "255356798")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "34005945")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "185410072")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "197415029")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "254468134")].index, inplace=True)



empresacontactoconvenio_PacientesSF.head()

In [ ]:
empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/SFpacientes.xlsx", index=False)

In [ ]:
contactd = sf.Account.get("0018c00002lUyo0AAC")
contactd["PersonContactId"]

In [ ]:
# contactd["PersonContactId"]
print(datetime.now())

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/tod  es.xlsx", index=False)

query2 = "select P_liza**c, Nombre_de_Empresa**c, Rut_Empresa**c, Empresa_Padre**r.Name, Empresa**r.name,Holding**c, Plan**c, Pais**c, Activo**c, Id from Datos_Empresa**c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()


Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)


derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId\_\_pc": "derty", "Email": "example@example.com"})


Subir_pacientes.head()


Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account\_\_l.xlsx", index=False)


In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()